In [2]:
'''
[Reference]
Tensorflow(2020.11) nmt_with_attention[Source Code]
https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/text/nmt_with_attention.ipynb
Building models for NMT
'''

### **Import Libraries**

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split 

import unicodedata
import re
import numpy as np
import os
import io
import time

### GPU setting

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13726352853067687357
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12775957749617462132
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11991246439
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8952938922944189331
physical_device_desc: "device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:02:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12889262038508714768
physical_device_desc: "device: XLA_GPU device"
]


### Data Load

In [3]:
path_to_file_esb = '../dataset/train/spa-eng/spa_for_esb.txt'

### Data Preprocessing

In [4]:
# 유니코드 파일을 아스키 코드 파일로 변환합니다.
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # 단어와 단어 뒤에 오는 구두점(.)사이에 공백을 생성합니다.
  # 예시: "he is a boy." => "he is a boy ."
  # 참고:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # (a-z, A-Z, ".", "?", "!", ",")을 제외한 모든 것을 공백으로 대체합니다.
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # 모델이 예측을 시작하거나 중단할 때를 알게 하기 위해서
  # 문장에 start와 end 토큰을 추가합니다.
  w = '<start> ' + w + ' <end>'
  return w

#### Dataset 생성
1. 문장에 있는 억양을 제거합니다.
2. 불필요한 문자를 제거하여 문장을 정리합니다.
3. 다음과 같은 형식으로 문장의 쌍을 반환합니다: [영어, 스페인어]

In [5]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

#### Language 가 들어오면 공백 단위로 토큰화
- fit_on_texts(): 문자 데이터를 입력받아서 리스트의 형태로 변환
- texts_to_sequences: 텍스트 안의 단어들을 숫자 시퀀스로 출력
- pad_sequcences(tensor, padding='post') : 서로 다른 개수의 단어로 이루어진 문장을 같은 길이로 만들어주기 위해 패딩을 사용
  - padding = 'post' : [[ 0  0  0  5  3  2  4], [ 0  0  0  5  3  2  7],...,]
  - padding = 'pre' : 뒤 부터 패딩이 채워짐
  - 가장 긴 sequence 의 길이 만큼
  

In [6]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

#### **전처리된 타겟 문장과 입력 문장 쌍을 생성**
- input_tensor : input 문장의 패딩 처리된 숫자 시퀀스
- inp_lang_tokenizer : input 문장을 공백 단위로 토큰화, 문자 -> 리스트 변환
- target_tensor, targ_lang_tokenizer : 위와 비슷


In [7]:
def load_dataset(path, num_examples=None):

  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

#### 언어 데이터셋 크기 제한
- 언어 데이터셋을 아래의 크기로 제한하여 훈련과 검증을 수행
- inp_lang, targ_lang : 인풋,타겟 문장의 문자 -> 리스트 변환 결과
- max_length_targ, max_length_inp : 인풋, 타겟 문장의 '패딩된' 숫자 시퀀스 길이 -> 타겟 텐서와 입력 텐서의 최대 길이

In [8]:
num_examples = 60000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file_esb, num_examples)

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

#### 데이터셋 (테스트 & 검증) 분리

In [9]:
# 훈련 집합과 검증 집합을 80대 20으로 분리합니다.
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# 훈련 집합과 검증 집합의 데이터 크기를 출력합니다.
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

48000 48000 12000 12000


#### 인덱스 -> 해당 word 로

```
Input Language; index to word mapping
1 ----> <start>
93 ----> tomas
27 ----> le
1063 ----> escribio
7 ----> a
120 ----> maria
3 ----> .
2 ----> <end>
```


```
Target Language; index to word mapping
1 ----> <start>
8 ----> tom
695 ----> wrote
6 ----> to
31 ----> mary
3 ----> .
2 ----> <end>
```

In [10]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

### Model Training

#### Buffer, Batch, epoch, embedding dimension, units 설정
- Tokenizer 의 word_index 속성 : 속성은 단어와 숫자의 키-값 쌍을 포함하는 딕셔너리를 반환
- 따라서 vocab_inp_size, vocab_inp_size : 인풋, 타겟의 단어-숫자 딕셔너리 최대 길이 + 1 (?)
- dataset.batch(BATCH_SIZE, drop_remainder = True) : 배치사이즈 만큼 분할 후 남은 데이터를 drop 할 것인지 여부
- shuffle : 데이터셋 적절히 섞어준다.

In [11]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [12]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 53]), TensorShape([64, 51]))

#### Encoder

1.   초기화 : vocab_size(단어의 크기), embedding_dim(임베딩 차원 수), enc_units(인코더의 히든 사이즈), batch_sz(배치 사이즈)
  - embedding_dim : 단어 -> 임베딩 벡터로 하기 위한 차원 수
2.  call : gru 에 들어가 output, state 출력
3.  initialize_hidden_state : 맨 처음 gru에 들어가기 위한 더미 입력 값

In [13]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

#### Encoder 객체 생성

In [14]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

#### Attention

In [15]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # 쿼리 은닉 상태(query hidden state)는 (batch_size, hidden size)쌍으로 이루어져 있습니다.
    # query_with_time_axis은 (batch_size, 1, hidden size)쌍으로 이루어져 있습니다.
    # values는 (batch_size, max_len, hidden size)쌍으로 이루어져 있습니다.
    # 스코어(score)계산을 위해 덧셈을 수행하고자 시간 축을 확장하여 아래의 과정을 수행합니다.
    query_with_time_axis = tf.expand_dims(query, 1)

    # score는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다.
    # score를 self.V에 적용하기 때문에 마지막 축에 1을 얻습니다.
    # self.V에 적용하기 전에 텐서는 (batch_size, max_length, units)쌍으로 이루어져 있습니다.
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다. 
    attention_weights = tf.nn.softmax(score, axis=1)

    # 덧셈이후 컨텍스트 벡터(context_vector)는 (batch_size, hidden_size)쌍으로 이루어져 있습니다.
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

#### Decoder

1.   초기화 : vocab_size(단어의 크기), embedding_dim(임베딩 차원 수), enc_units(인코더의 히든 사이즈), batch_sz(배치 사이즈)
2.   encoder 와의 차이점 : 마지막 fully_connected_layer(tf.keras.layers.Dense) 추가

In [16]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # 어텐션을 사용합니다.
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output는 (batch_size, max_length, hidden_size)쌍으로 이루어져 있습니다.
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # 임베딩층을 통과한 후 x는 (batch_size, 1, embedding_dim)쌍으로 이루어져 있습니다.
    x = self.embedding(x)

    # 컨텍스트 벡터와 임베딩 결과를 결합한 이후 x의 형태는 (batch_size, 1, embedding_dim + hidden_size)쌍으로 이루어져 있습니다.
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # 위에서 결합된 벡터를 GRU에 전달합니다.
    output, state = self.gru(x)

    # output은 (batch_size * 1, hidden_size)쌍으로 이루어져 있습니다.
    output = tf.reshape(output, (-1, output.shape[2]))

    # output은 (batch_size, vocab)쌍으로 이루어져 있습니다.
    x = self.fc(output)

    return x, state, attention_weights

#### Decoder 객체 생성

In [17]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

#### Loss Function & Optimizer

In [18]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

#### Chekcpoint
- 여기서 학습한 매개변수를 저장, optimizer/encoder/decoder

In [19]:
# 각각의 모델 번역 확인을 위해서 (각각 다른 모델 체크포인트)
checkpoint_dir = '../Checkpoint_super/training_checkpoints2'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

#### Train

In [20]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # 교사 강요(teacher forcing) - 다음 입력으로 타겟을 피딩(feeding)합니다.
    for t in range(1, targ.shape[1]):
      # enc_output를 디코더에 전달합니다.
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # 교사 강요(teacher forcing)를 사용합니다.
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [22]:
EPOCHS = 150

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # 에포크가 2번 실행될때마다 모델 저장 (체크포인트)
  # if (epoch + 1) % 2 == 0:
  #   checkpoint.save(file_prefix = checkpoint_prefix)
  checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.3641
Epoch 1 Batch 100 Loss 0.3282
Epoch 1 Batch 200 Loss 0.3358
Epoch 1 Batch 300 Loss 0.4030
Epoch 1 Batch 400 Loss 0.2950
Epoch 1 Batch 500 Loss 0.3961
Epoch 1 Batch 600 Loss 0.3444
Epoch 1 Batch 700 Loss 0.3490
Epoch 1 Loss 0.3331
Time taken for 1 epoch 237.2544116973877 sec

Epoch 2 Batch 0 Loss 0.2188
Epoch 2 Batch 100 Loss 0.2404
Epoch 2 Batch 200 Loss 0.2347
Epoch 2 Batch 300 Loss 0.2005
Epoch 2 Batch 400 Loss 0.2429
Epoch 2 Batch 500 Loss 0.2398
Epoch 2 Batch 600 Loss 0.2452
Epoch 2 Batch 700 Loss 0.2208
Epoch 2 Loss 0.2455
Time taken for 1 epoch 239.80951380729675 sec

Epoch 3 Batch 0 Loss 0.1664
Epoch 3 Batch 100 Loss 0.1472
Epoch 3 Batch 200 Loss 0.2362
Epoch 3 Batch 300 Loss 0.1789
Epoch 3 Batch 400 Loss 0.1870
Epoch 3 Batch 500 Loss 0.1885
Epoch 3 Batch 600 Loss 0.1873
Epoch 3 Batch 700 Loss 0.2284
Epoch 3 Loss 0.1844
Time taken for 1 epoch 239.42481112480164 sec

Epoch 4 Batch 0 Loss 0.1136
Epoch 4 Batch 100 Loss 0.1115
Epoch 4 Batch 200 Loss 0.116

Epoch 27 Batch 400 Loss 0.0143
Epoch 27 Batch 500 Loss 0.0143
Epoch 27 Batch 600 Loss 0.0158
Epoch 27 Batch 700 Loss 0.0171
Epoch 27 Loss 0.0133
Time taken for 1 epoch 240.39121079444885 sec

Epoch 28 Batch 0 Loss 0.0105
Epoch 28 Batch 100 Loss 0.0075
Epoch 28 Batch 200 Loss 0.0057
Epoch 28 Batch 300 Loss 0.0066
Epoch 28 Batch 400 Loss 0.0122
Epoch 28 Batch 500 Loss 0.0153
Epoch 28 Batch 600 Loss 0.0111
Epoch 28 Batch 700 Loss 0.0182
Epoch 28 Loss 0.0107
Time taken for 1 epoch 239.46493077278137 sec

Epoch 29 Batch 0 Loss 0.0080
Epoch 29 Batch 100 Loss 0.0086
Epoch 29 Batch 200 Loss 0.0058
Epoch 29 Batch 300 Loss 0.0171
Epoch 29 Batch 400 Loss 0.0154
Epoch 29 Batch 500 Loss 0.0216
Epoch 29 Batch 600 Loss 0.0119
Epoch 29 Batch 700 Loss 0.0170
Epoch 29 Loss 0.0145
Time taken for 1 epoch 239.05121898651123 sec

Epoch 30 Batch 0 Loss 0.0131
Epoch 30 Batch 100 Loss 0.0069
Epoch 30 Batch 200 Loss 0.0139
Epoch 30 Batch 300 Loss 0.0141
Epoch 30 Batch 400 Loss 0.0113
Epoch 30 Batch 500 Loss 0.0

Epoch 53 Batch 600 Loss 0.0159
Epoch 53 Batch 700 Loss 0.0080
Epoch 53 Loss 0.0078
Time taken for 1 epoch 239.40477657318115 sec

Epoch 54 Batch 0 Loss 0.0073
Epoch 54 Batch 100 Loss 0.0065
Epoch 54 Batch 200 Loss 0.0068
Epoch 54 Batch 300 Loss 0.0065
Epoch 54 Batch 400 Loss 0.0118
Epoch 54 Batch 500 Loss 0.0035
Epoch 54 Batch 600 Loss 0.0117
Epoch 54 Batch 700 Loss 0.0127
Epoch 54 Loss 0.0083
Time taken for 1 epoch 240.37216997146606 sec

Epoch 55 Batch 0 Loss 0.0080
Epoch 55 Batch 100 Loss 0.0054
Epoch 55 Batch 200 Loss 0.0100
Epoch 55 Batch 300 Loss 0.0116
Epoch 55 Batch 400 Loss 0.0102
Epoch 55 Batch 500 Loss 0.0048
Epoch 55 Batch 600 Loss 0.0088
Epoch 55 Batch 700 Loss 0.0068
Epoch 55 Loss 0.0088
Time taken for 1 epoch 239.10556173324585 sec

Epoch 56 Batch 0 Loss 0.0045
Epoch 56 Batch 100 Loss 0.0047
Epoch 56 Batch 200 Loss 0.0036
Epoch 56 Batch 300 Loss 0.0086
Epoch 56 Batch 400 Loss 0.0048
Epoch 56 Batch 500 Loss 0.0118
Epoch 56 Batch 600 Loss 0.0160
Epoch 56 Batch 700 Loss 0.0

Epoch 79 Loss 0.0081
Time taken for 1 epoch 240.33280634880066 sec

Epoch 80 Batch 0 Loss 0.0076
Epoch 80 Batch 100 Loss 0.0094
Epoch 80 Batch 200 Loss 0.0050
Epoch 80 Batch 300 Loss 0.0134
Epoch 80 Batch 400 Loss 0.0065
Epoch 80 Batch 500 Loss 0.0105
Epoch 80 Batch 600 Loss 0.0294
Epoch 80 Batch 700 Loss 0.0095
Epoch 80 Loss 0.0091
Time taken for 1 epoch 239.34475994110107 sec

Epoch 81 Batch 0 Loss 0.0113
Epoch 81 Batch 100 Loss 0.0053
Epoch 81 Batch 200 Loss 0.0044
Epoch 81 Batch 300 Loss 0.0131
Epoch 81 Batch 400 Loss 0.0083
Epoch 81 Batch 500 Loss 0.0111
Epoch 81 Batch 600 Loss 0.0103
Epoch 81 Batch 700 Loss 0.0088
Epoch 81 Loss 0.0093
Time taken for 1 epoch 239.39016819000244 sec

Epoch 82 Batch 0 Loss 0.0078
Epoch 82 Batch 100 Loss 0.0070
Epoch 82 Batch 200 Loss 0.0068
Epoch 82 Batch 300 Loss 0.0092
Epoch 82 Batch 400 Loss 0.0050
Epoch 82 Batch 500 Loss 0.0209
Epoch 82 Batch 600 Loss 0.0048
Epoch 82 Batch 700 Loss 0.0070
Epoch 82 Loss 0.0075
Time taken for 1 epoch 239.6295962333

Epoch 105 Loss 0.0063
Time taken for 1 epoch 238.4763104915619 sec

Epoch 106 Batch 0 Loss 0.0023
Epoch 106 Batch 100 Loss 0.0015
Epoch 106 Batch 200 Loss 0.0032
Epoch 106 Batch 300 Loss 0.0101
Epoch 106 Batch 400 Loss 0.0013
Epoch 106 Batch 500 Loss 0.0094
Epoch 106 Batch 600 Loss 0.0038
Epoch 106 Batch 700 Loss 0.0044
Epoch 106 Loss 0.0058
Time taken for 1 epoch 238.57191348075867 sec

Epoch 107 Batch 0 Loss 0.0049
Epoch 107 Batch 100 Loss 0.0026
Epoch 107 Batch 200 Loss 0.0105
Epoch 107 Batch 300 Loss 0.0096
Epoch 107 Batch 400 Loss 0.0067
Epoch 107 Batch 500 Loss 0.0080
Epoch 107 Batch 600 Loss 0.0072
Epoch 107 Batch 700 Loss 0.0082
Epoch 107 Loss 0.0064
Time taken for 1 epoch 239.02971601486206 sec

Epoch 108 Batch 0 Loss 0.0057
Epoch 108 Batch 100 Loss 0.0120
Epoch 108 Batch 200 Loss 0.0084
Epoch 108 Batch 300 Loss 0.0050
Epoch 108 Batch 400 Loss 0.0092
Epoch 108 Batch 500 Loss 0.0084
Epoch 108 Batch 600 Loss 0.0065
Epoch 108 Batch 700 Loss 0.0071
Epoch 108 Loss 0.0091
Time taken

Epoch 131 Batch 200 Loss 0.0044
Epoch 131 Batch 300 Loss 0.0081
Epoch 131 Batch 400 Loss 0.0087
Epoch 131 Batch 500 Loss 0.0043
Epoch 131 Batch 600 Loss 0.0103
Epoch 131 Batch 700 Loss 0.0123
Epoch 131 Loss 0.0083
Time taken for 1 epoch 239.3335976600647 sec

Epoch 132 Batch 0 Loss 0.0092
Epoch 132 Batch 100 Loss 0.0037
Epoch 132 Batch 200 Loss 0.0130
Epoch 132 Batch 300 Loss 0.0121
Epoch 132 Batch 400 Loss 0.0196
Epoch 132 Batch 500 Loss 0.0085
Epoch 132 Batch 600 Loss 0.0031
Epoch 132 Batch 700 Loss 0.0119
Epoch 132 Loss 0.0093
Time taken for 1 epoch 239.4116461277008 sec

Epoch 133 Batch 0 Loss 0.0050
Epoch 133 Batch 100 Loss 0.0030
Epoch 133 Batch 200 Loss 0.0076
Epoch 133 Batch 300 Loss 0.0084
Epoch 133 Batch 400 Loss 0.0079
Epoch 133 Batch 500 Loss 0.0054
Epoch 133 Batch 600 Loss 0.0109
Epoch 133 Batch 700 Loss 0.0101
Epoch 133 Loss 0.0072
Time taken for 1 epoch 238.97585487365723 sec

Epoch 134 Batch 0 Loss 0.0045
Epoch 134 Batch 100 Loss 0.0046
Epoch 134 Batch 200 Loss 0.0068
E

#### Translation(Spain -> English)

*   tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen='', padding='post') : 일정한 길이(maxlen)로 맞춰준다. (패딩은 뒤에서)
*   

  ```
  inp_lang.word_index :  {'<start>': 1, '<end>': 2, '.': 3, 'tom': 4, '?': 5...}
  ```

* tf.expand_dims: 차원을 늘려준다.




In [43]:
def evaluate(sentence):
#   attention_plot = np.zeros((max_length_targ, max_length_inp))

#   sentence = preprocess_sentence(sentence)

#   inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]

  # 문장, input 딕셔너리 출력 
  print ('sentence:', sentence)

  no_word = 'no word'

  sentence = preprocess_sentence(sentence)

  inputs = []
  for i in sentence.split(' '):
    
    if i in inp_lang.word_index:
        inputs.append(inp_lang.word_index[i])
    else:
        print('no word!')
        return no_word, sentence
    
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden

  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
    
  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
#       return result, sentence, attention_plot
      return result, sentence

    # 예측된 ID를 모델에 다시 피드합니다.
    dec_input = tf.expand_dims([predicted_id], 0)

#   return result, sentence, attention_plot
  return result, sentence

In [44]:
def translate(sentence):
#   result, sentence, attention_plot = evaluate(sentence)
  result, sentence = evaluate(sentence)
  
  return result
#   print('Input: %s' % (sentence))
#   print('Predicted translation: {}'.format(result))

In [45]:
# checkpoint_dir내에 있는 최근 체크포인트(checkpoint)를 복원합니다.
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [46]:
translate(u'El va a la escuela.')   # He is going to school.

sentence: El va a la escuela.


'he goes to school . <end> '

In [47]:
result = translate(u'El va a la escuela.')

sentence: El va a la escuela.


In [48]:
result

'he goes to school . <end> '

In [50]:
# test dataset 파일 불러오기
testdata = '../dataset/test/test_data.txt'
result_path = '../outputs/supermodel'
start_row = 2

# # 엑셀 파일 불러오기
# wb = openpyxl.load_workbook('TQE(5-Epoch).axlsx')
# 
# # 엑셀 파일의 시트 활성화
# sheet1 = wb['real']

f = open(testdata, 'r')
lines = f.readlines()
f.close()

f = open(result_path + '/hpys.txt', 'w')

for sentence in lines:
    result = translate(sentence)
    if result == 'no word':
        f.write('no word\n')
    else:
        f.write(result + '\n')
        
f.close()

sentence: 'Te dijeron lo que pasó, ¿no?'

sentence: 'Ellos no estaban allá.'

sentence: 'No me gusta ninguno de los chicos.'

sentence: 'Jamás trabajé con él.'

sentence: 'Mi padre cultiva arroz.'

sentence: 'Dime qué hiciste en Shounan.'

sentence: '¿Cuántas palabras deberías escribir?'

sentence: '¿Cómo te introdujiste en mi casa?'

sentence: 'Mi madre hornea pan todas las mañanas.'

sentence: 'Yo he disfrutado el leer esta novela.'

sentence: 'Él estaba parado en la esquina.'

sentence: 'El testigo no parecía estar nervioso cuando testificó en el juicio.'

sentence: '¿Dónde está el zoológico?'

sentence: 'El nombre de mi hijo es Tom.'

sentence: 'Tenemos cosas más grandes de qué preocuparnos.'

sentence: 'Juntémonos pasado mañana.'

sentence: '¿Qué estás escribiendo?'

sentence: '¿Este es tu vino?'

sentence: 'Era casi imposible circular por esa calle.'

no word!
sentence: 'Pensé que Tom querría ayudarme.'

sentence: '¿Qué le quieres regalar a Tom?'

sentence: 'No quiero permitir qu

sentence: '¿Por qué ella es tan popular?'

sentence: 'Dame un pedazo de papel para escribir, por favor.'

sentence: 'Para Hawking, escribir este libro no fue fácil.'

no word!
sentence: 'No sabía que supieras conducir.'

sentence: 'Casi todas las chicas son amables.'

sentence: 'Llevaré dos completos con mostaza y ketchup.'

no word!
sentence: 'No creo que Tom vaya a regresar.'

sentence: 'Tom, ¿qué quieres de cenar?'

sentence: 'Alrededor del año 174 a.C., su gente se volvió contra Roma.'

sentence: 'Kenia solía ser una colonia inglesa.'

sentence: '¿Qué tal estuvo la fiesta de Tom?'

sentence: 'Tom dijo que entiende cómo te sientes.'

sentence: 'Viven en la Tercera Avenida.'

sentence: 'Me sorprendió lo mucho que se parecía a su padre.'

sentence: 'Tom tendrá que esperar.'

sentence: 'Tom fue incapaz de disimular su sorpresa.'

no word!
sentence: 'Necesito a alguien con quien hablar.'

sentence: 'Estaba pensando en otra cosa.'

sentence: 'Tom finalmente se calmó.'

sentence: 'Necesit

sentence: 'Quiero que se vaya.'

sentence: 'No te muevas.'

sentence: '¿Por qué no estaban allí?'

sentence: 'No respondió al teléfono, así que le mandé un correo.'

sentence: 'Yo no te di un regalo.'

sentence: '¿Cómo suena eso?'

sentence: 'Esta noche vamos al cine.'

sentence: 'Vimos una momia en el museo.'

no word!
sentence: '¿En qué momento estudias?'

sentence: 'Ellos me miraban en silencio.'

no word!
sentence: 'Este es un gran proyecto.'

sentence: 'Él sabe hablar japonés.'

sentence: 'Existe un grado de estrés en todos los trabajos.'

sentence: 'Una catástrofe ha sido impedida.'

no word!
sentence: 'No sabía que hacer entonces.'

sentence: 'Me pregunto dónde se estará escondiendo.'

sentence: 'Estoy demasiado borracho.'

sentence: '¿Le falta a usted algo en su cartera?'

sentence: '¿Te gustaría un plátano?'

sentence: 'Repetí mi nombre.'

sentence: 'Ella quisiera que él se pusiera en contacto con ella tan pronto como sea posible.'

sentence: 'A todos le agradabas.'

no word!


sentence: 'Tras mucho reflexionar, el bandido decidió confesar.'

no word!
sentence: 'Él se giró.'

sentence: '¿No hay alguna otra cosa?'

sentence: '¿Es éste el autobús para Oxford?'

no word!
sentence: 'Tom ahora está en algún lugar en Australia.'

sentence: 'Deberías haber estado aquí.'

sentence: 'Tomás volvió a llamarla.'

sentence: 'La policia está buscando sospechosos.'

sentence: '¿Coméis carne?'

no word!
sentence: 'Yo soy de Noruega.'

sentence: 'Esto es lo mejor que se pone, chicos.'

sentence: 'Sabía que Tom no era bueno.'

sentence: 'Mañana voy a ir a verla.'

sentence: 'Eres un doctor.'

sentence: 'Tomás me estuvo mensajeando.'

no word!
sentence: 'Soñé que me estaba comiendo la tarta de bodas de mi nieta.'

sentence: 'Mañana voy a Boston.'

sentence: '¿Conoce Tom a Mary?'

sentence: '¿Qué libro necesita?'

sentence: 'Somos profesores.'

sentence: 'Yo soy el que recibió la paliza.'

sentence: 'Discúlpeme. No pensé que fuera su asiento.'

no word!
sentence: 'Ella siempre l

sentence: 'Te mostraré la ciudad.'

sentence: 'Debe su éxito a sus padres.'

sentence: 'Esa no es mi línea.'

sentence: 'Pagué la cuenta.'

sentence: 'Es hora de que los niños vayan a la cama.'

sentence: 'Tom bebe como un pez.'

sentence: 'Ella me saludó con una sonrisa.'

sentence: 'Todavía no hemos llamado a la policía.'

sentence: 'Adoptaron a la niña pequeña.'

sentence: 'Hervimos agua para hacer fideos.'

no word!
sentence: 'Tom no se da cuenta de lo afortunado que es.'

sentence: 'Ojalá hubiera aprendido esto en el colegio.'

sentence: 'Yo estuve aquí todo el tiempo.'

sentence: 'El bolígrafo está roto.'

sentence: 'Estaba parado detrás de la puerta.'

sentence: 'Tom miró a su alrededor frenéticamente.'

no word!
sentence: 'El tren paró en la estación.'

sentence: 'Puedo ser tu mejor amiga o tu peor enemiga.'

sentence: 'Ellos quieren saber cuál es tu plan.'

sentence: 'Tom era muy atractivo cuando era joven.'

sentence: 'Tom no tiene que pagar nada.'

sentence: 'Mantén tus mano

sentence: 'Mi padre me ayudó con los deberes.'

sentence: 'No estoy nada cansado.'

sentence: '¿Tiene seguro médico?'

sentence: 'Encontrar una solución óptima demorará un tiempo.'

no word!
sentence: 'Algunos estaban borrachos la mayor parte del tiempo.'

sentence: 'Cristóbal Colón descubrió América.'

sentence: 'A finales de agosto, las fuerzas aliadas tomaron París.'

no word!
sentence: 'Vendré a verte el domingo a las tres de la tarde.'

sentence: 'Quiero que seas mi socio.'

sentence: 'Ella es amable.'

sentence: 'No está de acuerdo con su familia.'

sentence: 'Está ocurriendo esta noche.'

sentence: 'Deberías haber llamado a la policía.'

sentence: 'Eres la última persona que habría pensado encontrar aquí.'

sentence: 'Esto no es un accidente.'

sentence: 'Habéis venido demasiado pronto.'

sentence: 'Dejó de buscar la perla.'

no word!
sentence: 'Le gusta cazar.'

sentence: 'No te olvides de despertarme mañana por la mañana.'

sentence: 'Tom decidió no decirle a María nada de lo 